In [ ]:
from tableau_api_lib import TableauServerConnection
from tableau_api_lib.utils import querying, flatten_dict_column
import io
from io import StringIO
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import xmltodict
import pandas as pd

import numpy as np
import pandas as pd
import re, gc
import time, requests
from datetime import datetime, timedelta
# import datetime
import calendar
from datetime import datetime, timedelta

import sqlalchemy as sql
from sqlalchemy.pool import NullPool
from urllib.parse import quote
# from tqdm import tqdm


from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, PatternFill, Border, Side


# Set display option to show all values without scientific notation
pd.set_option('display.float_format', '{:.2f}'.format)
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)


from openpyxl.styles import Font, PatternFill, Border, Side, Alignment
from openpyxl.styles import NamedStyle
import shutil
import os

# Declare Function

In [2]:

import datetime

def monthdelta(date, delta):
    m, y = (date.month + delta) % 12, date.year + ((date.month) + delta - 1) // 12
    if not m:
        m = 12
    d = min(date.day, [31, 29 if y % 4 == 0 and (not y % 100 == 0 or y % 400 == 0) else 28,
                       31, 30, 31, 30, 31, 31, 30, 31, 30, 31][m - 1])
    return date.replace(day=d, month=m, year=y)

today = datetime.datetime.today()

# Get the start period for the transaction (one month ago from today)
start_period = monthdelta(today, -1).replace(day=1).date()

# Get the end period for the transaction (first day of the next month)
end_period = monthdelta(today, 0).replace(day=1).date()

month_name = start_period.strftime("%B")  # Get the month name (e.g., June, July, etc.)
vPeriod = start_period.strftime("%Y%m")  # Format start_period directly
gvPeriod = start_period.strftime("%Y")   # Format only the year from start_period

print("start_period =", start_period.strftime("%Y-%m-%d"))
print("end_period =", end_period.strftime("%Y-%m-%d"))
print('month_name =', month_name)
print('vPeriod =', vPeriod)
print('gvPeriod =', gvPeriod)


In [3]:

from datetime import datetime, timedelta
import pandas as pd

def get_current_monday():
    today = datetime.now()
    days_until_monday = today.weekday()
    current_monday = today - timedelta(days=days_until_monday)
    return current_monday.strftime('%Y-%m-%d')

def get_previous_monday():
    today = datetime.now()
    days_until_monday = today.weekday()
    previous_monday = today - timedelta(days=days_until_monday + 7)
    return previous_monday.strftime('%Y-%m-%d')

# Get the dates for the current and previous Monday
this_monday = get_current_monday()
previous_monday = get_previous_monday()


In [4]:

# comma style uda bisa

# GOOD


# Define a function to apply styling to a worksheet
def apply_styling(ws):
    # Apply styling to the header row
    header_font = Font(bold=True)
    header_fill = PatternFill(start_color='D3D3D3', end_color='D3D3D3', fill_type='solid')
    header_border = Border(left=Side(border_style='thin', color='000000'),
                           right=Side(border_style='thin', color='000000'),
                           top=Side(border_style='thin', color='000000'),
                           bottom=Side(border_style='thin', color='000000'))

    for cell in ws['1']:
        cell.font = header_font
        cell.fill = header_fill
        cell.border = header_border
        cell.alignment = Alignment(wrap_text=True)  # Add text wrapping to header cells

    # Apply styling to the data cells
    data_font = Font()
    data_fill = PatternFill(start_color='EFEFEF', end_color='EFEFEF', fill_type='solid')
    data_border = Border(left=Side(border_style='thin', color='BFBFBF'),
                        right=Side(border_style='thin', color='BFBFBF'),
                        top=Side(border_style='thin', color='BFBFBF'),
                        bottom=Side(border_style='thin', color='BFBFBF'))

    # Create a custom style with number format
    number_format_style = NamedStyle(name='number_format_style', number_format='#,##0')

    for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
        for cell in row:
            if isinstance(cell.value, (int, float)):
                # Apply the custom style with number format
                cell.style = number_format_style
            cell.font = data_font
            cell.fill = data_fill
            cell.border = data_border
            cell.alignment = Alignment(wrap_text=True)  # Add text wrapping to data cells

    # Adjust column width to fit content
    for column in ws.columns:
        max_length = 0
        column = [cell for cell in column]
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column[0].column_letter].width = adjusted_width

    
    # Add AutoFilter to the worksheet
    ws.auto_filter.ref = ws.dimensions


In [5]:

def process_dataframe(df):
    # Select specific columns and rename them
    df = df.iloc[:, [0, 1, 2, 4, 5, 6, 7, 8, 9]].rename(columns={
        'Unnamed: 0': 'Sbu Group',
        'Unnamed: 1': 'Concept',
        'Unnamed: 2': 'Store Name',
        '# Member': 'Unique Member',
        '# Potential MAPCLUB Member': 'Potential MAPCLUB Member',
        '# Member Transaction': 'Member Transaction',
        '# Non Member Transaction': 'Non Member Transaction',
        'Total Sales': 'Member Sales',
        'Sales Non Member': 'Non Member Sales'
    })
    
    # Fill NaN values using forward fill
    df.fillna(method='ffill', inplace=True)
    
    return df

In [6]:

import shutil

def delete_folder(folder_path):
    try:
        shutil.rmtree(folder_path)
        print(f"The folder '{folder_path}' has been successfully deleted.")
    except OSError as e:
        print(f"Error: {e}")

# Example usage:
folder_to_delete = r'/opt/zeppelin/output_pm'
delete_folder(folder_to_delete)


In [7]:

# Create again table
folder_name = r'/opt/zeppelin/output_pm'

# Check if the folder already exists
if not os.path.exists(folder_name):
    # Create the folder
    os.makedirs(folder_name)
    print(f"The '{folder_name}' folder has been created.")
else:
    print(f"The '{folder_name}' folder already exists.")


# Configuration

In [9]:

config = {
    'tableau_prod_loyalty': {
        'server': 'https://visual.synectics.io/',
        'api_version': '3.19',
        'personal_access_token_name': 'zeppelin_task',
        'personal_access_token_secret': '4++/at5yT1+aNAgOPL6XDw==:RXO4TBaSNqrMl6faqTFP9bLQJIK9b31D',
        'site_name': 'Loyalty',
        'site_url': 'Loyalty'
    }
}
connection = TableauServerConnection(config_json=config, env='tableau_prod_loyalty')

connection.sign_in()



In [10]:

workbook_df = querying.get_workbooks_dataframe(conn=connection)
target_workbook_df = (
    workbook_df
    .where(workbook_df['name']=="Automation Potential Member")
    .dropna()
)

In [11]:
%python
#GET Auth API
url = "https://visual.synectics.io/api/3.19/auth/signin"

headers = {
    "Content-Type": "application/json",
}

data = {
    "credentials": {
        "name": "administrator",
        "password": "$",
        "site": {
            "contentUrl": "Loyalty"
        }
    }
}

response = requests.post(url, headers=headers, json=data)
# Parse XML to dictionary
xml_dict = xmltodict.parse(response.text)
#Store XML Value to variables
credentials_token = xml_dict['tsResponse']['credentials']['@token']
site_id = xml_dict['tsResponse']['credentials']['site']['@id']
content_url = xml_dict['tsResponse']['credentials']['site']['@contentUrl']
user_id = xml_dict['tsResponse']['credentials']['user']['@id']

In [12]:

target_view_df = querying.get_views_for_workbook_dataframe(
    conn=connection,
    workbook_id=target_workbook_df['id'].to_list().pop()
)

In [14]:

dashboard_id = (
        target_view_df
        .where(target_view_df['name']=='PLI')
        .dropna()
        .get('id').to_list().pop()
        )
url = "https://visual.synectics.io/api/3.14/sites/a2d01dc4-592a-4117-aa40-d023e5ccacb6/views/"+dashboard_id+"/crosstab/excel"
api_key = credentials_token
headers = {
    "Authorization": f"Bearer {api_key}"
}
response = requests.get(url, headers=headers)
df_PLI = pd.read_excel(pd.ExcelFile(io.BytesIO(response.content)))
df_PLI = process_dataframe(df_PLI)

file_name = (f"output_pm/Weekly_Potential_Member_-_PLI_{this_monday}.xlsx".format(this_monday=this_monday))
# Create a workbook and a worksheet
wb = Workbook()
ws = wb.active
# Add the DataFrame to the worksheet
for r in dataframe_to_rows(df_PLI, index=False, header=True):
    ws.append(r)
apply_styling(ws)

# Save the Excel file
wb.save(file_name)

print(f"Weekly_Potential_Member_-_PLI Excel file saved successfully.")

In [15]:

dashboard_id = (
        target_view_df
        .where(target_view_df['name']=='Active')
        .dropna()
        .get('id').to_list().pop()
        )
url = "https://visual.synectics.io/api/3.14/sites/a2d01dc4-592a-4117-aa40-d023e5ccacb6/views/"+dashboard_id+"/crosstab/excel"
api_key = credentials_token
headers = {
    "Authorization": f"Bearer {api_key}"
}
response = requests.get(url, headers=headers)

df_Active = pd.read_excel(pd.ExcelFile(io.BytesIO(response.content)))
df_Active = process_dataframe(df_Active)

file_name = (f"output_pm/Weekly_Potential_Member_-_Active_{this_monday}.xlsx".format(this_monday=this_monday))
# Create a workbook and a worksheet
wb = Workbook()
ws = wb.active
# Add the DataFrame to the worksheet
for r in dataframe_to_rows(df_Active, index=False, header=True):
    ws.append(r)
apply_styling(ws)

# Save the Excel file
wb.save(file_name)

print(f"Weekly_Potential_Member_-_Active Excel file saved successfully.")


In [16]:

dashboard_id = (
        target_view_df
        .where(target_view_df['name']=='Digimap')
        .dropna()
        .get('id').to_list().pop()
        )
url = "https://visual.synectics.io/api/3.14/sites/a2d01dc4-592a-4117-aa40-d023e5ccacb6/views/"+dashboard_id+"/crosstab/excel"
api_key = credentials_token
headers = {
    "Authorization": f"Bearer {api_key}"
}
response = requests.get(url, headers=headers)

df_Digimap = pd.read_excel(pd.ExcelFile(io.BytesIO(response.content)))
df_Digimap = process_dataframe(df_Digimap)

file_name = (f"output_pm/Weekly_Potential_Member_-_Digimap_{this_monday}.xlsx".format(this_monday=this_monday))
# Create a workbook and a worksheet
wb = Workbook()
ws = wb.active
# Add the DataFrame to the worksheet
for r in dataframe_to_rows(df_Digimap, index=False, header=True):
    ws.append(r)
apply_styling(ws)

# Save the Excel file
wb.save(file_name)

print(f"Weekly_Potential_Member_-_Digimap Excel file saved successfully.")


In [17]:

dashboard_id = (
        target_view_df
        .where(target_view_df['name']=='F&B')
        .dropna()
        .get('id').to_list().pop()
        )
url = "https://visual.synectics.io/api/3.14/sites/a2d01dc4-592a-4117-aa40-d023e5ccacb6/views/"+dashboard_id+"/crosstab/excel"
api_key = credentials_token
headers = {
    "Authorization": f"Bearer {api_key}"
}
response = requests.get(url, headers=headers)

df_FnB = pd.read_excel(pd.ExcelFile(io.BytesIO(response.content)))
df_FnB = process_dataframe(df_FnB)

file_name = (f"output_pm/Weekly_Potential_Member_-_F&B_{this_monday}.xlsx".format(this_monday=this_monday))
# Create a workbook and a worksheet
wb = Workbook()
ws = wb.active
# Add the DataFrame to the worksheet
for r in dataframe_to_rows(df_FnB, index=False, header=True):
    ws.append(r)
apply_styling(ws)

# Save the Excel file
wb.save(file_name)

print(f"Weekly_Potential_Member_-_Fnb Excel file saved successfully.")


In [18]:

dashboard_id = (
        target_view_df
        .where(target_view_df['name']=='Fashion')
        .dropna()
        .get('id').to_list().pop()
        )
url = "https://visual.synectics.io/api/3.14/sites/a2d01dc4-592a-4117-aa40-d023e5ccacb6/views/"+dashboard_id+"/crosstab/excel"
api_key = credentials_token
headers = {
    "Authorization": f"Bearer {api_key}"
}
response = requests.get(url, headers=headers)

df_Fashion = pd.read_excel(pd.ExcelFile(io.BytesIO(response.content)))
df_Fashion = process_dataframe(df_Fashion)

file_name = (f"output_pm/Weekly_Potential_Member_-_Fashion_{this_monday}.xlsx".format(this_monday=this_monday))
# Create a workbook and a worksheet
wb = Workbook()
ws = wb.active
# Add the DataFrame to the worksheet
for r in dataframe_to_rows(df_Fashion, index=False, header=True):
    ws.append(r)
apply_styling(ws)

# Save the Excel file
wb.save(file_name)

print(f"Weekly_Potential_Member_-_Fashion Excel file saved successfully.")


In [19]:

dashboard_id = (
        target_view_df
        .where(target_view_df['name']=='Foodhall')
        .dropna()
        .get('id').to_list().pop()
        )
url = "https://visual.synectics.io/api/3.14/sites/a2d01dc4-592a-4117-aa40-d023e5ccacb6/views/"+dashboard_id+"/crosstab/excel"
api_key = credentials_token
headers = {
    "Authorization": f"Bearer {api_key}"
}
response = requests.get(url, headers=headers)

df_Foodhall = pd.read_excel(pd.ExcelFile(io.BytesIO(response.content)))
df_Foodhall = process_dataframe(df_Foodhall)

file_name = (f"output_pm/Weekly_Potential_Member_-_Foodhall_{this_monday}.xlsx".format(this_monday=this_monday))
# Create a workbook and a worksheet
wb = Workbook()
ws = wb.active
# Add the DataFrame to the worksheet
for r in dataframe_to_rows(df_Foodhall, index=False, header=True):
    ws.append(r)
apply_styling(ws)

# Save the Excel file
wb.save(file_name)

print(f"Weekly_Potential_Member_-_Foodhall Excel file saved successfully.")


In [20]:

target_view_df

In [21]:

dashboard_id = (
        target_view_df
        .where(target_view_df['name']=='Travel')
        .dropna()
        .get('id').to_list().pop()
        )
url = "https://visual.synectics.io/api/3.14/sites/a2d01dc4-592a-4117-aa40-d023e5ccacb6/views/"+dashboard_id+"/crosstab/excel"
api_key = credentials_token
headers = {
    "Authorization": f"Bearer {api_key}"
}
response = requests.get(url, headers=headers)

df_Travel = pd.read_excel(pd.ExcelFile(io.BytesIO(response.content)))
df_Travel = process_dataframe(df_Travel)
# df_Travel
file_name = (f"output_pm/Weekly_Potential_Member_-_Travel_{this_monday}.xlsx".format(this_monday=this_monday))
# Create a workbook and a worksheet
wb = Workbook()
ws = wb.active
# Add the DataFrame to the worksheet
for r in dataframe_to_rows(df_Travel, index=False, header=True):
    ws.append(r)
apply_styling(ws)

# Save the Excel file
wb.save(file_name)

print(f"Weekly_Potential_Member_-_Travel Excel file saved successfully.")


In [22]:

# import pandas as pd

# Assuming you already have DataFrames named df_PLI, df_Active, etc.

# Create a tuple of DataFrames
sheets = (df_PLI, df_Active, df_Digimap, df_FnB, df_Fashion, df_Foodhall, df_Travel)

# Select .iloc[1:] for each DataFrame and concatenate them
combined_df = pd.concat([df.iloc[1:] for df in sheets], ignore_index=True)

# # Print the combined DataFrame
# # print(combined_df)
# combined_df.to_excel(f"output/Weekly_Potential_Member_-_All_SBU_{this_monday}.xlsx".format(this_monday=this_monday))

file_name = (f"output_pm/Weekly_Potential_Member_-_All_SBU_{this_monday}.xlsx".format(this_monday=this_monday))
# Create a workbook and a worksheet
wb = Workbook()
ws = wb.active
# Add the DataFrame to the worksheet
for r in dataframe_to_rows(combined_df, index=False, header=True):
    ws.append(r)
apply_styling(ws)

# Save the Excel file
wb.save(file_name)

print(f"Weekly_Potential_Member_-_All_SBU Excel file saved successfully.")


In [24]:

#running query

import os
import datetime
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

# Date formatting
today = datetime.date.today()
this_monday = today - datetime.timedelta(days=today.weekday())
previous_monday = this_monday - datetime.timedelta(days=7)

# Email configuration

sender_email = "dp-alert@notify.gtech.digital"
sender_password = "WTPQndoRvwc2i8"
receiver_email = "michel.hamilton@gtech.digital"
cc_receiver_email = "reyki.seprianza@gtech.digital, budi.santoso@mapclub.com, michael.danendra@mapclub.com, marcella.dewi@mapclub.com, rian.andrea@gtech.digital"

subject = "Weekly Potential Member-{}".format(this_monday)
# body = "Dear Pak Michel Hamilton, \nBerikut kami kirimkan Potential Dashboard untuk periode \nDari tanggal {} sampai dengan {}\n\n\n\n Testing kok guys".format(previous_monday,this_monday)
body = "Dear Pak Michel,\n\nKindly find the weekly email update (Store Performance) with the period of {} to {} on the attached files. \nThank you.\n\nBest regards,\n\nAutomatic System\n\n\n\n\n\n".format(previous_monday,this_monday)
# Excel attachment configuration
attachment_folder = '/opt/zeppelin/output_pm'

# Compose the email
msg = MIMEMultipart()
msg["From"] = sender_email
msg["To"] = receiver_email
msg["Subject"] = subject
msg["Cc"] = cc_receiver_email


msg.attach(MIMEText(body, "plain"))

# Attach all Excel files in the folder
for filename in os.listdir(attachment_folder):
    if filename.endswith(".xlsx"):
        file_path = os.path.join(attachment_folder, filename)
        with open(file_path, "rb") as f:
            attachment = MIMEApplication(f.read(), _subtype="xlsx")
            attachment.add_header("Content-Disposition", "attachment", filename=filename)
            msg.attach(attachment)

# Send the email
with smtplib.SMTP('smtpdm-ap-southeast-1.aliyun.com', 80) as server:
    server.login(sender_email, sender_password)
    server.send_message(msg)
    print("Email sent successfully!")
